# Team E - Pixels plus domain knowledge to True Shower parameters with CNNs

## Team Members

* Bjarne Karsten (3559535)
* Marek Freunscht (9604914)
* Florian Hartung (6622800)

## Data and Example Download
[...]

* an exemplary jupyter notebook and futher information is available from the git repo https://github.com/astrojarred/magic-ml-images
    * You find more information there. 


## TASK 

* Improve your understanding of the data 
* The focus of this teams is the setup of MLPs or CNNs and combining them with domain knowledge like the hillas parameters
* Use the pixel information features starting "clean_image" and the features starting with "hillas" and "stereo"
    * they contain information of all the pixels for a simulated particle
        * "clean_image" contains the data after some some standard cleaning algorithms were applied. These data are used in the MAGIC analysis to derive the Hillas Parameters.
        * "hillas" and "stereo" features are derived from the raw images by using the standard MAGIC analysis workflow. 
* Create a baseline using a simple ML algorithm to
    * classify gammas and protons 
        * by combining both clean_image_\* features
        * by combining both clean_image_\* features with "hillas" and/or "stereo" 
    * predict/infer the true shower parameters using a simple ML algorithm
        * using clean_image\_\*\_m1 and clean_image\_\*\_m2 independently
        * by combining both clean_image_\* features with "hillas" and/or "stereo" 
* Use MLP and simple CNN networks following the same strategy as above to
    * classify gammas and protons 
    * predict/infer true shower parameters using a simple ML algorithm
    * Think about a method to reuse the existing CNN kernels instead of using a hexagonal approch
* compare the different approaches 





## Implementation

* Start here with your implementation and documentation




In [ ]:
# add your implementation (code/documentation) here! 